In [ ]:
# ==========================================
# STEP 1: INSTALL DEPENDENCIES
# ==========================================
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

# **Step A: Load Model & Tokenizer**

In [ ]:
from unsloth import FastLanguageModel, tokenizer_utils
import torch

# --- 1. THE MONKEY PATCH (The "Hacker" Fix) ---
# We overwrite the internal function that validates the template.
# This prevents it from raising the RuntimeError about the missing prompt.
def strict_fix_chat_template_bypass(tokenizer):
    # We do nothing but return the tokenizer and None (acting as success)
    return tokenizer, None

# Apply the patch to the library at runtime
tokenizer_utils.fix_chat_template = strict_fix_chat_template_bypass
print("✅ Unsloth validation check bypassed via Monkey Patch.")
# ----------------------------------------------

max_seq_length = 1024
dtype = None
load_in_4bit = True

print("⏳ Loading SeaLLMs-v3-7B-Chat...")

# Now this will run without the RuntimeError OR the TypeError
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "SeaLLMs/SeaLLMs-v3-7B-Chat",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    fix_tokenizer = False,
)

# --- 2. Apply Your Template Fix ---
# Now we manually set the correct template as you intended
tokenizer.chat_template = """{% if messages[0]['role'] == 'system' %}
    {% set loop_messages = messages[1:] %}
    {% set system_message = messages[0]['content'] %}
{% else %}
    {% set loop_messages = messages %}
    {% set system_message = "You are a helpful assistant." %}
{% endif %}
{{ system_message }}
{% for message in loop_messages %}
    {% if message['role'] == 'user' %}
        {{ 'User: ' + message['content'] + '\n' }}
    {% elif message['role'] == 'assistant' %}
        {{ 'Assistant: ' + message['content'] + '\n' }}
    {% endif %}
{% endfor %}
{% if add_generation_prompt %}
    {{ 'Assistant: ' }}
{% endif %}"""

print("✅ Tokenizer Template Manually Applied!")

# --- 3. Initialize LoRA ---
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

print("🚀 Success! Model loaded and adapters initialized.")

✅ Unsloth validation check bypassed via Monkey Patch.
⏳ Loading SeaLLMs-v3-7B-Chat...
==((====))==  Unsloth 2025.11.6: Fast Qwen2 patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Tokenizer Template Manually Applied!


Unsloth 2025.11.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


🚀 Success! Model loaded and adapters initialized.


# STEP 3: PREPARE DATASET (Khmer Logic)

In [ ]:
# ==========================================
# STEP 3: PREPARE DATASET (Khmer Logic)
# ==========================================
from datasets import Dataset
import json

# Define the Prompt Template (Same as your Qwen/DeepSeek model for consistency)
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def detect_subject_and_task(text):
    text_lower = text.lower()
    subjects = {
        "Biology": ["adn", "dna", "នុយក្លេអូទីត", "សែន", "ប្រូតេអ៊ីន", "biology", "អរម៉ូន", "ឈាម"],
        "Physics": ["ម៉ូទ័រ", "កម្ដៅ", "អគ្គិសនី", "physics", "សូលេណូអ៊ីត", "រលក", "ម៉ាញេទិច", "voltage", "current"],
        "Chemistry": ["សូលុយស្យុង", "ph", "អាស៊ីត", "chemistry", "សមីការ", "ម៉ូល", "ប្រតិកម្ម", "ch3"],
        "History": ["សង្គមរាស្ត្រនិយម", "history", "ឯករាជ្យ", "ខ្មែរក្រហម", "បារាំង", "សីហនុ"],
        "Earth Science": ["earth science", "សិលា", "ដី", "សំណឹក", "ផែនដី"]
    }
    detected_subject = "General Knowledge"
    for subject, keywords in subjects.items():
        if any(k in text_lower for k in keywords):
            detected_subject = subject
            break

    is_generation_request = any(x in text_lower for x in ["generate", "create", "give me", "i need a"])
    return detected_subject, is_generation_request

formatted_data = []
file_path = "rean_ai_exercise_generation_no_year.jsonl"

try:
    print(f"📖 Reading dataset from {file_path}...")
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            entry = json.loads(line)
            messages = entry["messages"]
            user_content = messages[0]["content"]
            assistant_content = messages[1]["content"]

            subject, is_gen_request = detect_subject_and_task(user_content)

            if is_gen_request:
                instruction = user_content
                input_text = ""
            else:
                instruction = f"សូមដោះស្រាយលំហាត់ {subject} ខាងក្រោម។ (Solve the following {subject} exercise.)"
                input_text = user_content

            text = alpaca_prompt.format(instruction, input_text, assistant_content) + tokenizer.eos_token
            formatted_data.append({"text": text})

    dataset = Dataset.from_list(formatted_data)

    # Split the dataset into training and evaluation sets
    train_dataset = dataset.train_test_split(test_size=0.1, seed=42)
    eval_dataset = train_dataset['test']
    train_dataset = train_dataset['train']

    print(f"✅ Loaded {len(dataset)} examples. Split into {len(train_dataset)} training and {len(eval_dataset)} evaluation examples.")

except FileNotFoundError:
    print(f"❌ Error: Please upload '{file_path}' to Colab files first!")


📖 Reading dataset from rean_ai_exercise_generation_no_year.jsonl...
✅ Loaded 333 examples. Split into 299 training and 34 evaluation examples.


In [ ]:
# ==========================================
# STEP 4: TRAIN THE MODEL (FIXED)
# ==========================================
from trl import SFTTrainer
from transformers import TrainingArguments

print("🚀 Starting Training...")

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        output_dir = "outputs",

        # --- THE FIX ---
        eval_strategy = "steps", # Renamed from 'evaluation_strategy'
        # ----------------

        eval_steps = 5,
    ),
)

trainer.train()

🚀 Starting Training...


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/299 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/34 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 299 | Num Epochs = 2 | Total steps = 60
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176 of 7,655,986,688 (0.53% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


...

Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
5,1.062000,1.199008
10,0.957900,0.953758
15,0.795800,0.791064
20,0.734800,0.684731
25,0.662900,0.624877
30,0.632800,0.587123
35,0.526600,0.550982
40,0.470800,0.530503
45,0.427400,0.504648
50,0.345900,0.482309


Unsloth: Not an error, but Qwen2ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


eval/loss,█▆▄▃▃▂▂▂▁▁▁▁
eval/runtime,█▁▁▁▂▂▁▁▂▂▁▁
eval/samples_per_second,▁███▇▇██▇▇██
eval/steps_per_second,▁███▇▇██▇▇██
train/epoch,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇███
train/global_step,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇███
train/grad_norm,▁▂▄▆▆█▄▄▅▅█▇
train/learning_rate,▇█▇▇▆▅▄▄▃▂▂▁
train/loss,█▇▅▅▄▄▃▂▂▁▁▁
eval/loss,0.45644
eval/runtime,18.9457


TrainOutput(global_step=60, training_loss=0.6107917447884877, metrics={'train_runtime': 1183.824, 'train_samples_per_second': 0.405, 'train_steps_per_second': 0.051, 'total_flos': 8684267253322752.0, 'train_loss': 0.6107917447884877, 'epoch': 1.588628762541806})

In [ ]:
# ==========================================
# STEP 5: SAVE & DOWNLOAD ADAPTER
# ==========================================
import shutil
from google.colab import files

folder_name = "khmer_seallm_adapter"

# Save locally in Colab
print("💾 Saving Adapters...")
model.save_pretrained(folder_name)
tokenizer.save_pretrained(folder_name)

# Zip for download
print(f"📦 Zipping '{folder_name}'...")
shutil.make_archive(folder_name, 'zip', folder_name)

# Trigger Download
print("⬇️ Downloading to your computer...")
files.download(f"{folder_name}.zip")

💾 Saving Adapters...
📦 Zipping 'khmer_seallm_adapter'...
⬇️ Downloading to your computer...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os
print("restarting session...")
os.kill(os.getpid(), 9)